# Create adatom defects in a slab material

Create an adatom by specifying the chemical element, approximate position on surface and distance z, which will be resolved to:
- the equidistant position between the closes atoms on the surface according to Voronoi tesselation, 
- or the crystal site of the next layer that is closest to specified position.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Set defects parameters in cell 2.1. (or use default).
1. Click “Run” > “Run All” to run all cells. 
1. Wait for the run to complete (depending on the parameters can take a few min). 
1. Scroll down to view results. 

## Summary
1. Prepare the Environment: Set up the notebook and install packages, preview the input materials
1. Create the Defect: Add an adatom defect to the slab material
2. Visualize the Defect: Visualize the defect structure

## Notes

1. For more information, see [Introduction](Introduction.ipynb)
<!-- # TODO: use a hashtag-based anchor link to interface creation documention above -->


## 1. Prepare the Environment
### 1.1. Set up defect parameters 

In [ ]:
DEFECT_TYPE = "adatom"  
APPROXIMATE_POSITION_ON_SURFACE = [0.5, 0.5]  # Position of the defect in crystal coordinates
DISTANCE_Z = 2.0  # Distance of the defect from the surface in Angstrom
CHEMICAL_ELEMENT = "Si"  # Element to be placed at the site 
MILLER_INDICES = (1, 1, 1)  # Miller indices of the surface
SLAB_THICKNESS = 3  # Thickness of the slab in unit cells
VACUUM = 6  # Vacuum thickness in Angstrom
SUPERCELL_MATRIX = [[2, 0, 0], [0, 2, 0], [0, 0, 1]] 

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("create_point_defect.ipynb", "../../config.yml")

### 1.3. Get input material and 
Materials are loaded with `get_data()`. The first material is assigned as substrate and the second as film.

In [ ]:
from mat3ra.made.material import Material
from utils.jupyterlite import get_data

# Get the list of input materials and load them into `materials_in` variable
get_data("materials_in", globals())
materials = list(map(Material, globals()["materials_in"]))

### 1.4. Create and preview Slab

In [ ]:
from mat3ra.made.tools.build.slab import SlabConfiguration, get_terminations, create_slab
from utils.visualize import visualize_materials as visualize

material = materials[0]
slab_config = SlabConfiguration(
        bulk=material,
        miller_indices=MILLER_INDICES,
        thickness=SLAB_THICKNESS,
        vacuum=VACUUM,
        use_orthogonal_z=True,
        xy_supercell_matrix=SUPERCELL_MATRIX
    )
termination = get_terminations(slab_config)[0]
slab = create_slab(slab_config, termination)
visualize([{"material":slab , "rotation":"0x"}, {"material": slab, "rotation": "-90x"}],repetitions=[1, 1, 1])

## 2. Create the Defect
### 2.1. Set adatom parameters

In [ ]:
from mat3ra.made.tools.build.defect import AdatomSlabPointDefectConfiguration
from mat3ra.made.tools.build.defect.builders import EquidistantAdatomSlabDefectBuilder, CrystalSiteAdatomSlabDefectBuilder

adatom_config = AdatomSlabPointDefectConfiguration(crystal=slab, 
                                              defect_type=DEFECT_TYPE, 
                                              chemical_element=CHEMICAL_ELEMENT, 
                                              distance_z=DISTANCE_Z, 
                                              position_on_surface=APPROXIMATE_POSITION_ON_SURFACE
)

### 2.2. Create the adatom

In [ ]:
from mat3ra.made.tools.build.defect import create_slab_defect
slab_with_adatom_at_specified_position = create_slab_defect(adatom_config)
slab_with_adatom_at_equidistant_position = create_slab_defect(adatom_config, EquidistantAdatomSlabDefectBuilder())
slab_with_adatom_at_crystal_site = create_slab_defect(adatom_config, CrystalSiteAdatomSlabDefectBuilder())

## 3. Visualize the Slabs with Adatom

In [ ]:
from utils.visualize import visualize_materials as visualize

visualize([{"material": slab, "title": "Original material"},
           {"material": slab_with_adatom_at_equidistant_position, "title": f"Material with adatom defect at equidistant position"},
           {"material": slab_with_adatom_at_crystal_site, "title": f"Material with adatom defect at crystal site"}],
          rotation="-90x"
          )
visualize([{"material": slab, "title": "Original material"},
           {"material": slab_with_adatom_at_equidistant_position, "title": f"Material with adatom defect at equidistant position"},
           {"material": slab_with_adatom_at_crystal_site, "title": f"Material with adatom defect at crystal site"}]
)

## 4. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import set_data

set_data("materials", [slab_with_adatom_at_equidistant_position.to_json(), slab_with_adatom_at_crystal_site.to_json()])